In [365]:
import h5py
import numpy as np
import capnp
import sys
import os.path
import glob
sys.path.append('/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt')

import CapnpGame_capnp

In [366]:
raw_data_folder = '/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/*'
train_val_path = '/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/train_val.h5'

# probability of a specific game is chosen for the validation set
val_prob = 0.05 
boardsize = 9

# states the dataset can hold, changes only if file at train_val_path does not exist
samples = 25000

In [380]:
try: # open if dataset already exists
    dataset = h5py.File(train_val_path, 'r+', libver='latest')
    train_x = dataset['train_x']
    train_y = dataset['train_y']
    val_x = dataset['val_x']
    val_y = dataset['val_y']
except OSError:
    # create file for dataset
    dataset = h5py.File(train_val_path, 'w', libver='latest')
    
    #dataset.swmr_mode = True # not sure if we want that
    train_x = dataset.create_dataset("train_x", shape=(int(samples*(1-val_prob)),12,boardsize,boardsize), dtype='int8')
    train_y = dataset.create_dataset("train_y", shape=(int(samples*(1-val_prob)),83))
    val_x = dataset.create_dataset("val_x", shape=(int(samples*val_prob),12,boardsize,boardsize), dtype='int8')
    val_y = dataset.create_dataset("val_y", shape=(int(samples*val_prob),83))
    
    # save where to (over-)write next in dataset
    train_x.attrs.create("next_i_to_overwrite",0)
    val_x.attrs.create("next_i_to_overwrite",0)

In [394]:
def write_in_dataset(force_full_write=False,total_written=[0,0]):
    """
    TODO: make code nice and readable
    TODO: writing should stop once we see the first entry was already written 
    at some other time / only write new entries
    force_full_write = True will make sure that there will be no empty entries 
    in the dataset by duplicates some datapoints
    total_written = states written in [train_set,val_set]
    """
    paths = glob.glob(raw_data_folder)
    paths.reverse()
    print(paths)
    for raw_data_path in paths:
        print(raw_data_path)
        raw_data = h5py.File(raw_data_path,'r')
        for i in range(raw_data['game_record'].attrs['count_id']):
            print(i)
            #print(raw_data['game_record'][i].tostring())
            game_msg = raw_data['game_record'][i].tostring()
            #print(game_msg)
            g = CapnpGame_capnp.Game.from_bytes(game_msg)
            
            if np.random.sample(1)[0] > val_prob and total_written[0] < train_x.shape[0]:
                dest_x, dest_y, j = train_x, train_y, 0
            elif total_written[1] < val_x.shape[0]:
                dest_x, dest_y, j = val_x, val_y, 1
            else:
                raw_data.close()
                return
                
            for i in range(len(g.stateprobs)):
                offset = dest_x.attrs["next_i_to_overwrite"]
                dest_x[offset] = np.array(g.stateprobs[i].state).reshape((12 , boardsize, boardsize))

                probs = np.array(g.stateprobs[i].probs)
                winner = np.asarray([int(g.result)])

                y = np.concatenate((probs.flatten(), winner), axis=0)

                dest_y[offset] = y.reshape((1,83))

                dest_x.attrs.modify("next_i_to_overwrite",(offset+1)%dest_x.shape[0])

                total_written[j] += 1
                if total_written[j] > dest_x.shape[0]:
                    print("total_written exeeded dest.shape")
                    break
                
            # save to disk after every iteration(?)
            dataset.flush()
        raw_data.close()
    if force_full_write:
        write_in_dataset(force_full_write=True,total_written=total_written)
    return
    

In [396]:
write_in_dataset(force_full_write=False)

['/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0002.h5', '/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0001.h5']
/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0002.h5
0
1
/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0001.h5
0
1


In [300]:
list(raw_data.keys())

['game_record']

In [315]:
raw_data.close
paths = glob.glob(raw_data_folder)
paths.reverse()

for raw_data_path in paths:
    raw_data = h5py.File(raw_data_path,'r')
    for file in raw_data['game_record']:
        print(file)

[6 0 0 ..., 0 0 0]
[0 0 0 ..., 0 0 0]
[3 0 0 ..., 0 0 0]
[4 0 0 ..., 0 0 0]
[3 0 0 ..., 0 0 0]
[3 0 0 ..., 0 0 0]


In [349]:
list(raw_data.keys())

['game_record']

In [352]:
raw_data['game_record'].attrs['count_id']

2

In [379]:
train_x

<Closed HDF5 dataset>

In [378]:
dataset = h5py.File(train_val_path, 'r+', libver='latest')

In [393]:
paths = glob.glob(raw_data_folder)
paths.reverse()
print(paths)
for raw_data_path in paths:
    print(raw_data_path)

['/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0002.h5', '/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0001.h5']
/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0002.h5
/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/raw_data/game_record-0001.h5


In [397]:
dataset.close()